# Common Libraries

In [25]:
import gzip
import re
import xml.etree.ElementTree as ET
import pandas as pd

# Params

In [28]:
brodman_nifti_path = "/usr/local/afni/abin/Brodmann_pijn_afni.nii.gz"
save_xml_data_path = "/home/seojin/Seojin_commonTool/Module/Brain_ROI/metadata.xml"

is_save_xml = True
save_info_path = "/home/seojin/Seojin_commonTool/Module/Brain_ROI/Brodmann_pijn_afni_info.csv"

# Load data

In [3]:
with gzip.open(brodman_nifti_path, 'rt', errors='ignore') as f:
    content = f.read()

# Filtering meta data

In [4]:
metadata = re.findall(r'<AFNI_attributes.*?</AFNI_attributes>', content, re.DOTALL)[0]

if is_save_xml:
    with open(save_xml_data_path, 'w') as xml_file:
        xml_file.write(metadata)

# Read meta data

In [6]:
root = ET.fromstring(metadata)
value_label_dtable = None
for atr in root.findall(".//AFNI_atr"):
    if atr.get('atr_name') == "VALUE_LABEL_DTABLE":
        value_label_dtable = atr.text
        break

'\n "<VALUE_LABEL_DTABLE\nni_type="2*String"\nni_dimen="78"\npbar_name="ROI_i256">\n"1" "ctx-lh-BA1_3"\n"2" "ctx-lh-BA2"\n"3" "ctx-lh-BA4"\n"4" "ctx-lh-BA5"\n"5" "ctx-lh-BA6"\n"6" "ctx-lh-BA7"\n"7" "ctx-lh-BA8"\n"8" "ctx-lh-BA9"\n"9" "ctx-lh-BA10"\n"10" "ctx-lh-BA11"\n"11" "ctx-lh-BA13"\n"12" "ctx-lh-BA16"\n"13" "ctx-lh-BA17"\n"14" "ctx-lh-BA18"\n"15" "ctx-lh-BA19"\n"16" "ctx-lh-BA20"\n"17" "ctx-lh-BA21"\n"18" "ctx-lh-BA22"\n"19" "ctx-lh-BA23"\n"20" "ctx-lh-BA24"\n"21" "ctx-lh-BA25"\n"22" "ctx-lh-BA26_29_30"\n"23" "ctx-lh-BA27"\n"24" "ctx-lh-BA28"\n"25" "ctx-lh-BA31"\n"26" "ctx-lh-BA32"\n"27" "ctx-lh-BA33"\n"28" "ctx-lh-BA34"\n"29" "ctx-lh-BA35_36"\n"30" "ctx-lh-BA37"\n"31" "ctx-lh-BA38"\n"32" "ctx-lh-BA39"\n"33" "ctx-lh-BA40"\n"34" "ctx-lh-BA41_42_52"\n"35" "ctx-lh-BA43"\n"36" "ctx-lh-BA44"\n"37" "ctx-lh-BA45"\n"38" "ctx-lh-BA46"\n"39" "ctx-lh-BA47"\n"101" "ctx-rh-BA1_3"\n"102" "ctx-rh-BA2"\n"103" "ctx-rh-BA4"\n"104" "ctx-rh-BA5"\n"105" "ctx-rh-BA6"\n"106" "ctx-rh-BA7"\n"107" "ctx-rh-

# Filtering roi info

In [14]:
labels = []
descriptions = []
for row in value_label_dtable.split("\n"):
    datas = row.split(" ")
    n_str = len(datas)

    if n_str > 1:
        label = datas[0].strip('"')
        description = datas[1].strip('"')
        
        if ("ctx" in description) or ("BA" in description):
            labels.append(label)
            descriptions.append(description)

In [17]:
empty_index = labels.index("")
descriptions[empty_index-1] = descriptions[empty_index - 1] + descriptions[empty_index]

In [18]:
del labels[empty_index]
del descriptions[empty_index]

# Save data

In [31]:
save_info = pd.DataFrame({ "label" : labels, "description" : descriptions })
save_info.to_csv(save_info_path, index=False)

In [33]:
save_info_path

'/home/seojin/Seojin_commonTool/Module/Brain_ROI/Brodmann_pijn_afni_info.csv'